<a href="https://colab.research.google.com/github/chris-lovejoy/youtube-titles-and-transcripts/blob/main/%5Creproduce%5Ccreating_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install ijson
import ijson
import csv

     |████████████████████████████████| 133kB 5.7MB/s 


First download raw_caption.json, HowTo100M_v1.csv, task_ids.csv from https://www.di.ens.fr/willow/research/howto100m/ and video_titles-en.csv (link in README) \\

Add paths to the files specified below as strings:

In [ ]:
captions = '...' # raw_caption json which can be downloaded from https://www.di.ens.fr/willow/research/howto100m/
titles = '...' # video_titles-en.csv
HowTo100M_v1 = '...' # HowTo100M_v1.csv which can be downloaded from https://www.di.ens.fr/willow/research/howto100m/
taskids = '...' # 'task_ids.csv' which can be downloaded from https://www.di.ens.fr/willow/research/howto100m/
full_dataset = '...' # A blank csv to put dataset in

In [ ]:
# Remove NaNs (3 different ways in which they appear)

f=open(captions,'r+')

chunksize=1000000
overlap=100
currentposition=0

while True:
    chunk = f.read(chunksize+overlap)
    f.seek(currentposition)
    if chunk == '':
        break
    nanstring = '[NaN, "'     
    repstring = '[" ", "'
    if chunk.find(nanstring) != -1:
        print('found one!')
        replacestring = chunk.replace(nanstring,repstring)
        f.write(replacestring)
    currentposition += chunksize
    f.seek(currentposition)

print('done')

In [ ]:
f=open(captions,'r+')

chunksize=1000000
overlap=100
currentposition=0

while True:
    chunk = f.read(chunksize+overlap)
    f.seek(currentposition)
    if chunk == '':
        break
    nanstring = ', NaN,'
    repstring = ', " ",'
    if chunk.find(nanstring) != -1:
        print('found one!')
        replacestring = chunk.replace(nanstring,repstring)
        f.write(replacestring)
    currentposition += chunksize
    f.seek(currentposition)
    
print('done')  

In [ ]:
f=open(captions,'r+')

chunksize=1000000
overlap=100
currentposition=0

while True:
    chunk = f.read(chunksize+overlap)
    f.seek(currentposition)
    if chunk == '':
        break
    nanstring = ', NaN]'     
    repstring = ', " "]'
    if chunk.find(nanstring) != -1:
        print('found one!')
        replacestring = chunk.replace(nanstring,repstring)
        f.write(replacestring)
    currentposition += chunksize
    f.seek(currentposition)
    
print('done')  

# Creating csv

In [ ]:
# Create dictionary of titles

with open(titles, 'r') as title_file:
  read = csv.reader(title_file)
  headers = next(read) # skip header

  TitleDictionary = {}

  for row in read:
    key = row[0]
    value = row[2]
    
    # Add to dictionary 
    TitleDictionary[key] = value  

In [ ]:
TitleDictionary['I5uqJ-MIZZQ']
# len(TitleDictionary)
# TitleDictionary

'Fixing American Girl Doll Kits Hair!/ How To Wash Short Doll Hair! AGLF'

In [ ]:
# Create dictionary of categories

with open(HowTo100M_v1, 'r') as cat_file:
  read = csv.reader(cat_file)
  headers = next(read) # skip header

  Cat1Dictionary = {}
  Cat2Dictionary = {}

  for row in read:
    key = row[0]
    cat1 = row[1]
    cat2 = row[2]
    
    #vAdd to dictionary 
    Cat1Dictionary[key] = cat1
    Cat2Dictionary[key] = cat2

In [ ]:
# Create dictionary of task ids

# Map task id number to name of task
with open(taskids, 'r') as taskid_file:
  read = csv.reader(taskid_file, delimiter='\t')

  TaskIDDictionary1 = {}

  for row in read:
    key = row[0]
    value = row[1]
    
    # Add to dictionary 
    TaskIDDictionary1[key] = value

# Map video id to task id number which maps to name of task
with open(HowTo100M_v1, 'r') as title_file:
  read = csv.reader(title_file)
  headers = next(read)

  TaskIDDictionary2 = {}

  for row in read:
    key = row[0]
    value = row[4]
    
    TaskIDDictionary2[key] = TaskIDDictionary1[value]

In [ ]:
# Put transcripts into csv
import csv

# Helper function to only add non-overlapping part of new caption to current transcript
def add_string_no_overlap(string1, string2):
  k = 0
  for i in range(5, len(string2)): # Choose 5 as lower end of range otherwise consecutive captions which happen to end/start with the same character(s) are wrongly concatenated
      if string1.endswith(string2[:i]):
          k = i
  return string1+' '+string2[k:]

data = ijson.parse(open(captions, 'rb'))

i=0 # Counter
curr_vid_ID = ''

with open(full_dataset, mode='w') as transcript_csv:
  # Create column titles
  fieldnames = ['vid_id', 'title', 'transcript', 'category_1', 'category_2', 'task_id']
  writer = csv.DictWriter(transcript_csv, fieldnames=fieldnames)
  writer.writeheader()
  for prefix, event, value in data:
    if event == 'string':
      # If prefix (video ID) hasn't changed, add this caption to current transcript
      if prefix == curr_vid_ID:
        # This code helps to deal with overlapping captions
        value = value.replace("\n", " ").replace("   ", " ").replace("  ", " ")
        transcript = add_string_no_overlap(transcript, value)
        transcript = transcript.replace("  ", " ")
        text_vid_ID = prefix
      else:
        # If reached new video, add previous transcript to csv and start next transcript
        if i > 0: # If not the first iteration
          transcript = transcript.replace("\n", " ")
          try:
            writer.writerow({'vid_id': text_vid_ID[:11], 'title': TitleDictionary[text_vid_ID[:11]], 'transcript': transcript, 'category_1': 
                             Cat1Dictionary[text_vid_ID[:11]], 'category_2': Cat2Dictionary[text_vid_ID[:11]], 
                             'task_id': TaskIDDictionary2[text_vid_ID[:11]]})
          except:
            None
        transcript = value.replace("\n", " ").replace("   ", " ").replace("  ", " ")
        i+=1

    curr_vid_ID = prefix

# Add punctuated flag

In [ ]:
import pandas as pd
import string

df = pd.read_csv(full_dataset)

# Adding punctuation flag

df['punctuated1'] = df['transcript'].str.contains(', ')
df['punctuated2'] = df['transcript'].str.contains('. ', regex=False)

df['punctuated'] = df['punctuated1'] & df['punctuated2']
df = df.drop(columns=['punctuated1', 'punctuated2'])

df.to_csv(full_dataset,index=False)

In [ ]:
df.punctuated.value_counts()

False    1063137
True       96964
Name: punctuated, dtype: int64

In [ ]:
df

,vid_id,title,transcript,category_1,category_2,task_id,punctuated
0,GhOxhZeJp74,Dermatology Treatments : How to Get Rid of Bac...,hi i'm dr. rafael Darvish with skin packable d...,Youth,Personal Care for Youth,Naturally Get Rid of Back Acne,False
1,igF53XMINhs,"""Norwood"" Rainbow Loom Bracelet/How To Tutorial",hey guys it's loves to loom here and today I'm...,Hobbies and Crafts,Crafts,Make a Loopy Bracelet,False
2,mt2DcsJFqAY,How to set up a wired game controller to your pc,are those me again stay I'm back from video an...,Food and Entertaining,Recipes,Make Game Controller Cookies,False
3,nxj62Bl2rrs,Pokemon go cookies - Easy Pokeball cookies NO ...,hello welcome Marlin here today we're making s...,Food and Entertaining,Recipes,Make a Pokéball,False
4,nVbIUDjzWY4,HOW TO PAINT YOUR MOTORCYCLE PART#5 BASECOAT PREP,other folks that Roylott v-twins to be AIDS I'...,Cars & Other Vehicles,Motorcycles,Paint a Motorcycle,False
...,...,...,...,...,...,...,...
1160096,8h7K6E_BUok,How to Make a Hover Ball Toy Easy | Fun DIY wi...,[Music] hi guys when you hear it's DIY time we...,Hobbies and Crafts,Crafts,Make Cute Hearts from Plastic Straws,False
1160097,tVc0qvCeVfQ,How to make a Tiny Paper Sail boat for scrapbo...,hello everyone its owner from honest paper gra...,Hobbies and Crafts,Crafts,Make a Sail,False
1160098,gIURSGIXX0Y,How to make a styrofoam RC airplane yourself |...,[Music] today I'm going to teach you how to bu...,Hobbies and Crafts,Crafts,Make a Flying Model Plane from Scratch,False
1160099,gHTVgMEpR2Q,#1 Yacon Molasses-90 Day Money Back Guarantee,hey everyone about two years ago I switched j...,Food and Entertaining,Recipes,Make Molasses Coady,False


# Adding additional columns to dataset (transcript length, default language etc.)

## Transcript length

In [ ]:
df['transcript_num_chars']  = df['transcript'].str.len()

In [ ]:
df = df.loc[df['transcript_num_chars'] <= 42500]

## Default Language & Default Audio Language

In [ ]:
# Create dictionary of titles
import csv

with open(titles, 'r') as title_file:
  read = csv.reader(title_file)
  headers = next(read) # skip header

  LangDict = {}
  AudioLangDict = {}

  for row in read:
    key = row[0]
    lang = row[3]
    audio_lang = row[4]
    
    LangDict[key] = lang
    AudioLangDict[key] = audio_lang  

In [ ]:
df['default_language'] = df['vid_id'].map(LangDict)
df['default_audio_language'] = df['vid_id'].map(AudioLangDict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.to_csv(full_dataset,index=False)